# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [2]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [4]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [5]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms'] * sales['bedrooms']

#in the dataset, 'floors' was defined with type string,
#so we will convert them to float, before creating a new feature
sales['floors'] = sales['floors'].astype(float)
sales['floors_square'] = sales['floors'] * sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [6]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [7]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=1e10) 

Find what features had non-zero weight.

** 3. Quiz Question: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights? **

In [8]:
print 'number of nonzeros = %d' % (model_all.coefficients['value']).nnz()
w_list = list(model_all.coefficients['value'])
for w in w_list:
    if not w: continue
    print w

number of nonzeros = 6
274873.05595
8468.53108691
24.4207209824
350.060553386
842.068034898
20.0247224171


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [9]:
model_all.coefficients.print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

** Be *very* careful that you use seed = 1 to ensure you get the same answer! **

In [10]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [11]:
RSS_list = []
for l1_penalty in np.logspace(1, 7, num=13):
    lasso_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    print 'l1_penalty = %e' % l1_penalty
    predict_price = lasso_model.predict(validation)
    term = predict_price- validation['price']
    RSS = (term * term).sum()
    RSS_list.append(RSS)
    print 'l1_penalty = %e, RSS = %f' % (l1_penalty, RSS)

print 'minimum RSS: %f' % min(RSS_list)

l1_penalty = 1.000000e+01
l1_penalty = 1.000000e+01, RSS = 625766285142459.875000
l1_penalty = 3.162278e+01
l1_penalty = 3.162278e+01, RSS = 625766285362394.125000
l1_penalty = 1.000000e+02
l1_penalty = 1.000000e+02, RSS = 625766286057885.000000
l1_penalty = 3.162278e+02
l1_penalty = 3.162278e+02, RSS = 625766288257224.625000
l1_penalty = 1.000000e+03
l1_penalty = 1.000000e+03, RSS = 625766295212186.750000
l1_penalty = 3.162278e+03
l1_penalty = 3.162278e+03, RSS = 625766317206080.500000
l1_penalty = 1.000000e+04
l1_penalty = 1.000000e+04, RSS = 625766386760658.125000
l1_penalty = 3.162278e+04
l1_penalty = 3.162278e+04, RSS = 625766606749278.500000
l1_penalty = 1.000000e+05
l1_penalty = 1.000000e+05, RSS = 625767302791634.125000
l1_penalty = 3.162278e+05
l1_penalty = 3.162278e+05, RSS = 625769507643886.250000
l1_penalty = 1.000000e+06
l1_penalty = 1.000000e+06, RSS = 625776517727024.000000
l1_penalty = 3.162278e+06
l1_penalty = 3.162278e+06, RSS = 625799062845467.000000
l1_penalty = 1.0

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`? 10^1
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [12]:
lasso_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=10.0)
predict_price_test= lasso_model.predict(testing)
term_test = predict_price_test- testing['price']
RSS_test = (term * term).sum()
print RSS_test

nonzero = (lasso_model.coefficients['value']).nnz()
print nonzero

6.25883719085e+14
18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [13]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [14]:
l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [15]:
nonzeros_dict = dict()
for l1_penalty in l1_penalty_values:
    lasso_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    nonzero = (lasso_model.coefficients['value']).nnz()
    nonzeros_dict[l1_penalty] = nonzeros_dict.get(l1_penalty, nonzero)
    print 'l1_penalty = %e, nonzeros = %d' % (l1_penalty, nonzero)
    print lasso_model.coefficients['value']


l1_penalty = 1.000000e+08, nonzeros = 18
[25090.917367212027, 7789.177061100565, 847.5596869428746, 25234.209194458632, 39.03944596356858, 1117.3118955675666, -0.025686118239919543, 143.98899196976993, 20695.35923964388, 12466.690650291855, 568204.6445841154, 91066.94280879853, 6360.780926249808, 6139.2128056478905, 43.035829924605586, 118.94587495435255, 9.040401654020448, 48.61546730927457]
l1_penalty = 1.274275e+08, nonzeros = 18
[26746.661936635508, 7743.979047849886, 822.358945250938, 25178.625930586946, 39.010718135253995, 1114.9107159232667, -0.018663073722782936, 142.51979784137154, 20545.867304697553, 12339.245250245602, 558930.2470718035, 90439.72185124886, 6288.009465538855, 6118.412320616764, 42.95755676813044, 117.99004260104351, 8.924085433195646, 46.56185624767168]
l1_penalty = 1.623777e+08, nonzeros = 18
[28873.181016622384, 7691.047075686332, 790.9175796843559, 25115.27853451593, 38.98207881320865, 1112.2394146479912, -0.02473736058080104, 140.9458447508174, 20365.2658

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

In [16]:
l1_penalty_min = 2.976351e+09 
l1_penalty_max = 3.792690e+09

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [17]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [18]:
for l1_penalty in l1_penalty_values:
    lasso_model_narrow = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    nonzero = (lasso_model_narrow.coefficients['value']).nnz()
    predict_price_narrow = lasso_model_narrow.predict(validation)
    term = predict_price_narrow- validation['price']
    RSS = (term * term).sum()
    RSS_list.append(RSS)
    print 'l1_penalty = %f, RSS = %f, non_zeros = %d' % (l1_penalty, RSS, nonzero)
    lasso_model_narrow.coefficients.print_rows(num_rows = 18)


l1_penalty = 2976351000.000000, RSS = 966925608279538.250000, non_zeros = 10
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 196100.908463 |  None  |
|     bedrooms     |  None | 2181.57568643 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 17962.6984763 |  None  |
|   sqft_living    |  None | 34.1424670927 |  None  |
| sqft_living_sqrt |  None | 789.319874138 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None | 3665.93448461 |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |  11333.858388 |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 3578.90100398 |  None  |
|    

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?


**  l1_penalty = 3448968315.789474, RSS = 1046937444590445.500000, non_zeros = 7 **

**
```
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.164946 |  None  |
|     bedrooms     |  None | 661.724576016 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9598946 |  None  |
|   sqft_living    |  None | 32.4102237547 |  None  |
| sqft_living_sqrt |  None |  690.11489972 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42111297 |  None  |
|    sqft_above    |  None | 30.0115786986 |  None  |
|  sqft_basement   |  None |      0.0      |  None  |
|     yr_built     |  None |      0.0      |  None  |
|   yr_renovated   |  None |      0.0      |  None  |
+------------------+-------+---------------+--------+
[18 rows x 4 columns]
```
**